In [1]:
import numpy as np
eps = np.finfo(float).eps

In [43]:
rows = 5
cols = 5
n = rows*cols
ditch_h = np.zeros((rows, cols))
ditch_h[:,2] = -0.5
ditch_h[2,:] = -0.7
print(ditch_h)

H = np.random.uniform(10, 13, (rows, cols))
ele = np.random.uniform(13, 16, (rows, cols))

print(H)
print('')
print(ele)

[[ 0.   0.  -0.5  0.   0. ]
 [ 0.   0.  -0.5  0.   0. ]
 [-0.7 -0.7 -0.7 -0.7 -0.7]
 [ 0.   0.  -0.5  0.   0. ]
 [ 0.   0.  -0.5  0.   0. ]]
[[10.32754453 11.4135972  11.4719317  10.61477445 11.99916532]
 [11.67719753 10.6409024  10.63790619 12.91839413 10.0084918 ]
 [10.72170052 12.78701202 11.16383711 11.25178101 11.01600384]
 [11.89807664 11.43828081 11.05229015 11.05668966 12.5788164 ]
 [10.3930705  10.06792842 12.09742316 12.1863083  12.1457976 ]]

[[14.50635758 15.72862515 14.22710383 15.57192242 14.95333061]
 [14.99484794 14.31229175 14.77263966 13.03590223 14.68497657]
 [15.86326737 15.27492815 14.16860711 15.55395219 13.34099601]
 [13.8572232  15.33174643 14.17144929 14.95060421 15.17454268]
 [15.5972728  15.16233471 15.61307866 15.7686704  14.23255054]]


In [44]:
ditch_h = np.ravel(ditch_h)
H = np.ravel(H)
ele = np.ravel(ele)

In [45]:
H_neighbours = ditch_h.copy()
for k in np.where(ditch_h < -eps)[0]:
    H_ave = 0
    n_neigh = 0
    if k%cols != 0 and ditch_h[k-1] > -eps: # west non-ditch neighbor
            H_ave += H[k-1]
            n_neigh += 1
    if (k+1)%cols != 0 and ditch_h[k+1] > -eps: # east non-ditch neighbor
            H_ave += H[k+1]
            n_neigh += 1
    if k-cols >= 0 and  ditch_h[k-cols] > -eps: # north non-ditch neighbor
            H_ave += H[k-cols]
            n_neigh += 1
    if k+cols < n and ditch_h[k+cols] > -eps: # south non-ditch neighbor
            H_ave += H[k+cols]
            n_neigh += 1
    if n_neigh > 0:
        H_neighbours[k] = H_ave / n_neigh  # average of neighboring non-ditch nodes
    else:  # corners or nodes surrounded by ditches dont have neighbors, given its ditch depth
        H_neighbours[k] = ele[k] + ditch_h[k] + eps

In [46]:
H = np.reshape(H, (rows, cols))
H_neighbours = np.reshape(H_neighbours, (rows, cols))
print(H_neighbours)

[[ 0.          0.         11.01418582  0.          0.        ]
 [ 0.          0.         11.77964826  0.          0.        ]
 [11.78763708 11.0395916  13.46860711 11.98754189 11.2936541 ]
 [ 0.          0.         11.24748523  0.          0.        ]
 [ 0.          0.         11.12711836  0.          0.        ]]


In [50]:
print(H)

[[12.49734315 11.55418889 11.42248859 12.24223044 11.74507153]
 [11.92804839 10.43148664 11.18436487 11.09015742 10.33270065]
 [11.61340788 10.73840725 11.7621676  11.76574752 12.15024899]
 [10.13573128 11.73197784 12.14564975 12.23236203 12.30202147]
 [10.08716767 12.56317282 11.18905776 10.0659173  11.18255857]]


In [62]:
(H[1,4] + H[3,4]) / 2

11.317361060433047

In [14]:
# Identify ditches (where ditch_h < -eps)
ditch_mask = ditch_h < -eps

# Create an array to store averaged values
H_neighbours = ditch_h.copy()

# Shifted versions of H to access neighboring values
H_west  = np.roll(H, shift=1, axis=1)  # Left shift
H_east  = np.roll(H, shift=-1, axis=1) # Right shift
H_north = np.roll(H, shift=1, axis=0)  # Up shift
H_south = np.roll(H, shift=-1, axis=0) # Down shift

# Mask for valid non-ditch neighbors (to prevent wrapping)
valid_west  = np.roll(ditch_mask, shift=1, axis=1) == False
valid_east  = np.roll(ditch_mask, shift=-1, axis=1) == False
valid_north = np.roll(ditch_mask, shift=1, axis=0) == False
valid_south = np.roll(ditch_mask, shift=-1, axis=0) == False

# Sum up valid neighbor values
H_ave = (
    H_west * valid_west +
    H_east * valid_east +
    H_north * valid_north +
    H_south * valid_south
)

# Count valid neighbors
n_neigh = valid_west.astype(int) + valid_east.astype(int) + valid_north.astype(int) + valid_south.astype(int)

# Compute the average only where there are valid neighbors
avg_mask = (ditch_mask & (n_neigh > 0))
H_neighbours[avg_mask] = (H_ave / n_neigh)[avg_mask]

# Assign ditch depth where there are no valid neighbors
H_neighbours[ditch_mask & (n_neigh == 0)] = ele[ditch_mask & (n_neigh == 0)] + ditch_h[ditch_mask & (n_neigh == 0)] + eps


/var/folders/01/f14lxpfj5wl28zjtf5rz8t9c0000gn/T/ipykernel_40287/4018311415.py:32: RuntimeWarning: invalid value encountered in divide
  H_neighbours[avg_mask] = (H_ave / n_neigh)[avg_mask]


In [15]:
H_neighbours

array([[ 0.        ,  0.        , 11.11027366, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 12.12287699, ...,  0.        ,
         0.        ,  0.        ],
       [11.37664706, 11.75838548, 12.69679265, ..., 10.99436821,
        10.59409334, 11.84028402],
       ...,
       [ 0.        ,  0.        , 11.49692606, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 11.10939738, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , 12.45997164, ...,  0.        ,
         0.        ,  0.        ]])

In [47]:
TrW1 = np.ravel(np.random.uniform(10, 13, (rows, cols)))
TrE1 = np.ravel(np.random.uniform(10, 13, (rows, cols)))
TrN1 = np.ravel(np.random.uniform(10, 13, (rows, cols)))
TrS1 = np.ravel(np.random.uniform(10, 13, (rows, cols)))
alfa = 1.334
implic = 0.5

In [48]:
# Setup of diagonal sparse matrix
a_d = implic * (TrW1 + TrE1 + TrN1 + TrS1) + alfa  # Diagonal
a_w = -implic * TrW1[1:]  # West element
a_e = -implic * TrE1[:-1]  # East element
a_n = -implic * TrN1[cols:]  # North element
a_s = -implic * TrS1[:n-cols]  # South element

In [49]:
cols

5